In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import numpy as np
import torchvision
from torchvision import transforms
import os
from torch import nn

In [8]:
import warnings

### Data PipeLine 만들기

In [5]:
df = pd.read_csv("/opt/ml/input/data/train/train.csv")
df.head()

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59


In [6]:
age_df = df.copy()
age_df.head()

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59


### 53 ~ 57 제거하기

In [7]:
age_df = age_df[~age_df['age'].isin([53, 54, 55, 56, 57])]
age_df.head()

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59
5,000007,female,Asian,58,000007_female_Asian_58


In [9]:
warnings.filterwarnings('ignore')

### image 경로와 label 붙이기

In [11]:
root = '/opt/ml/input/data/train/images/'
data_list = pd.DataFrame(None, columns = ['Id', 'path', 'label'])
for path in df['path']:
    Id, gender, race, age = path.split("_")
    age = int(age)
    age_point = 0
    if age < 30:
        age_point = 0
    elif age < 58:
        age_point = 1
    else:
        age_point = 2
        
    gender_point = 0
    if gender == "male":
        gender_point = 0
    else:
        gender_point = 3
        
    for image in list(os.listdir(os.path.join(root, path))):
        if image[0] == ".":
            continue
        if image.split('.')[0] == 'normal':
            label = 12 + age_point + gender_point
            data = {'Id' : Id ,'path' : os.path.join(f'{root}',  path, f'{image}'), 'label' : label}
            data_list = data_list.append(data, ignore_index = True)
        elif image.split('.')[0] == 'incorrect_mask':
            label = 6 + age_point + gender_point
            data = {'Id' : Id ,'path' : os.path.join(f'{root}', path, f'{image}'), 'label' : label}
            data_list = data_list.append(data, ignore_index = True)
        elif 'mask' in image.split('.')[0]:
            label = age_point + gender_point
            data = {'Id' : Id ,'path' : os.path.join(f'{root}',  path, f'{image}'), 'label' : label}
            data_list = data_list.append(data, ignore_index = True)
            
data_list.tail(3)
            


,Id,path,label
18897,006959,/opt/ml/input/data/train/images/006959_male_As...,0
18898,006959,/opt/ml/input/data/train/images/006959_male_As...,0
18899,006959,/opt/ml/input/data/train/images/006959_male_As...,0


### Mask, Gender, Age 별로 label 붙이기

In [12]:
mask_dict = {}
gender_dict = {}
age_dict = {}
for i in range(18):
    if i % 3 == 0:
        age_dict[i] = 0
    elif i % 3 == 1:
        age_dict[i] = 1
    else:
        age_dict[i] = 2
        

for i in range(18):
    if i <= 5:
        mask_dict[i] = 0
    elif i <= 11:
        mask_dict[i] = 1
    else:
        mask_dict[i] = 2


for i in range(18):
    if i <= 2 or (6 <= i <= 8) or (12 <= i <= 14):
        gender_dict[i] = 0
    else:
        gender_dict[i] = 1
data_list['Gender'] = data_list['label'].map(gender_dict)
data_list['Mask'] = data_list['label'].map(mask_dict)
data_list['Age'] = data_list['label'].map(age_dict)

In [13]:
data_list.tail(3)

,Id,path,label,Gender,Mask,Age
18897,006959,/opt/ml/input/data/train/images/006959_male_As...,0,0,0,0
18898,006959,/opt/ml/input/data/train/images/006959_male_As...,0,0,0,0
18899,006959,/opt/ml/input/data/train/images/006959_male_As...,0,0,0,0


In [ ]:
data_list.ifn